In [1]:
import os
import csv
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Attention, Conv1D, Conv2D
from keras.metrics import MeanSquaredError
from keras.losses import MeanSquaredError

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
model_name = "lru"

In [3]:
def read_df(path, embeddings_path=None):
    # Read csv
    df = pd.read_csv(path)
    
    # Append filename and source_file column column
    filenames = []
    for i in df["id"]:
        nmb = i.split('_')[-1]
        filenames.append(i[:-len(nmb)-1])
    df["file_name"] = filenames
    filenames = list(set(filenames))
    df['source_file'] = df["file_name"].apply(lambda x: int(x[:3]))
    # Append embeddings
    if embeddings_path != None:
        embedding_df = pd.read_csv(embeddings_path)
        embedding_df['source_file'] = embedding_df['fname']
        embedding_df['embeddings'] = embedding_df['embeddings'].apply(lambda x: np.array([*map(float, x.strip('][').replace("'","").split(','))]))
        df = pd.merge(df, embedding_df).drop(columns=["fname"])
    # Calculate miss ratios into a seperate dataframe
    miss_ratios = pd.DataFrame()
    for i in ["1024","2048","4096","8192"]:
        miss_ratios[i] = df['misses_'+i] / df['accesses_'+i]
        df = df.drop(columns=['misses_'+i, 'accesses_'+i])
        if 'l1d_accesses_'+i in df.columns:
            df = df.drop(columns=['l1d_accesses_'+i])
    return df, miss_ratios
rds, mrs = read_df('/kaggle/input/cache-estimator-data/output_512/dataset_lru.csv', '/kaggle/input/cache-estimator-data/embeddings/flow-aware-f.csv')
print(rds.shape, mrs.shape)

(6163, 900) (6163, 4)


In [4]:
rds.head

<bound method NDFrame.head of                          id         0         1         2        3        4  \
0       400.perlbench-41B_0  50816880  17323312  11868112  7701840  3853504   
1     400.perlbench-41B_436  17361520   6023600   4203744  2591488  1828656   
2       400.perlbench-50B_0  58283520  17249296  14592592  9466576  7204896   
3          401.bzip2-226B_0   3098304   3037808   2698720  2692176  1999568   
4         401.bzip2-226B_54   1623056   1618144   1501200  1353728  1014736   
...                     ...       ...       ...       ...      ...      ...   
6158     657.xz_s-3167B_466   4869744   5968224   3301024  2821488  2218352   
6159     657.xz_s-3167B_574   4967104   7056944   3530208  2928336  2367040   
6160       657.xz_s-4994B_0  18063488  19199008   9437200  7269680  7833904   
6161     657.xz_s-4994B_343  13498912  14343616   6633152  5098416  6450640   
6162         657.xz_s-56B_0  26404208  36893024  10084752  2060848  2511312   

            5        

There's a chance that reuse distance histograms within files are similar to each other. This may be due to them being sampled from the same files and therefore generated from the same code, so the histograms will be similar. To avoid this interfering with our process, we'll be moving all the lines that are derived from randomly picked files until they add up to 20% of the dataset. 

In [5]:
# 18 embeddings
files = ['482.sphinx3-1297B', '482.sphinx3-1522B', '482.sphinx3-1100B', '482.sphinx3-1395B', '482.sphinx3-234B', '482.sphinx3-417B', '600.perlbench_s-570B', '600.perlbench_s-1273B', '600.perlbench_s-210B', '470.lbm-1274B', '621.wrf_s-6673B', '621.wrf_s-8065B', '621.wrf_s-575B', '621.wrf_s-8100B', '400.perlbench-50B', '400.perlbench-41B', '403.gcc-48B', '403.gcc-16B', '403.gcc-17B', '436.cactusADM-1804B', '464.h264ref-57B', '464.h264ref-64B', '464.h264ref-30B', '464.h264ref-97B', '459.GemsFDTD-765B', '459.GemsFDTD-1418B', '459.GemsFDTD-1211B', '459.GemsFDTD-1320B', '459.GemsFDTD-1169B', '459.GemsFDTD-1491B', '435.gromacs-111B', '435.gromacs-134B', '435.gromacs-228B', '435.gromacs-226B', '625.x264_s-18B', '625.x264_s-33B', '625.x264_s-39B', '625.x264_s-12B', '625.x264_s-20B', '458.sjeng-767B', '458.sjeng-1088B', '458.sjeng-283B', '458.sjeng-31B', '473.astar-153B', '473.astar-42B', '473.astar-359B', '453.povray-576B', '453.povray-887B', '453.povray-252B', '453.povray-800B', '627.cam4_s-490B', '627.cam4_s-573B', '450.soplex-92B', '450.soplex-247B', '434.zeusmp-10B', '429.mcf-51B', '429.mcf-217B', '429.mcf-192B', '429.mcf-22B', '429.mcf-184B']
# files = []
def train_test_split(df, miss_ratios, split=0.3):
    filenames = list(set(df["source_file"]))
    collected_rows = 0
    source_file_ctr = 0
    global files
    if len(files) == 0:
        # Fill 2/3 of the test-set will with completely new to the model embeddings
        while len(df) * split * 2 / 3 > collected_rows:
            rf = filenames[random.randint(0,len(filenames)-1)]
            tmp = True
#             cs = ['file_name', 'source_file']
#             print(rf, df[df["source_file"] == rf][cs])
            for fnm in set(df[df["source_file"] == rf]['file_name']):
#                 print(fnm, fnm in files)
                if fnm in files:
                    tmp = False
                    break
                files.append(fnm)
                collected_rows += len(df[df["file_name"] == fnm])
            source_file_ctr += tmp
#             if tmp:
#                 print(rf)
#                 print(files)

        first_files = len(files)
        filenames = list(set(df["file_name"]))
        print(f"{source_file_ctr} embeddings (out of a 47) have been moved from the train set with their respective {first_files} files.")
        print(f"and their {collected_rows} rows")
        # The rest will just new files. 
        while len(df) * split > collected_rows:
            rf = filenames[random.randint(0,len(filenames)-1)]
            if rf in files: # No duplicates
                continue
            files.append(rf)
            collected_rows += len(df[df["file_name"] == rf])
        print(files)
    train_rds = df[~df["file_name"].isin(files)]
    train_mrs = miss_ratios[~df["file_name"].isin(files)]
    test_rds = df[df["file_name"].isin(files)]
    test_mrs = miss_ratios[df["file_name"].isin(files)]
    print(f"The test set consists of {len(files)} files (out of 189) with a total of {len(test_rds)} rows (out of {len(df)})")
    test_rds[["id", "file_name", "source_file"]].to_csv(f'/kaggle/working/id_col_{model_name}.csv')
    train_rds = train_rds.drop(columns=["id", "file_name", "source_file"])
    test_rds = test_rds.drop(columns=["id", "file_name", "source_file"])
    return train_rds, train_mrs, test_rds, test_mrs
train_rds, train_mrs, test_rds, test_mrs  = train_test_split(rds, mrs)
print(train_rds.shape)

The test set consists of 60 files (out of 189) with a total of 1943 rows (out of 6163)
(4220, 897)


In [6]:
path = '/kaggle/input/cache-estimator-data/output_512/dataset_lru.csv'               # Path to the dataset
embeddings_path = '/kaggle/input/cache-estimator-data/embeddings/flow-aware-f.csv'   # Path to the embeddings
rds, mrs = read_df(path, embeddings_path)
train_rds, train_mrs, test_rds, test_mrs = train_test_split(rds, mrs, split=0.25)

if 'embeddings' in train_rds.columns:
    x = []
    for i in train_rds['embeddings']:
        x.append(i)
    train_embeddings = np.array(x)
    train_rds = train_rds.drop(columns=["embeddings"])
    
    x = []
    for i in test_rds['embeddings']:
        x.append(i)
    test_embeddings = np.array(x)
    test_rds = test_rds.drop(columns=["embeddings"])
    
    embedding_scaler = StandardScaler()
    embedding_scaler = embedding_scaler.fit(train_embeddings)
    # train_embeddings = embedding_scaler.transform(train_embeddings)
    # test_embeddings  = embedding_scaler.transform(test_embeddings)
scaler = StandardScaler()
scaler = scaler.fit(train_rds)
# train_rds = scaler.transform(train_rds)
# test_rds = scaler.transform(test_rds)


The test set consists of 60 files (out of 189) with a total of 1943 rows (out of 6163)


In [7]:
train_embeddings.shape, test_embeddings.shape, train_embeddings.shape[0] + test_embeddings.shape[0]

((4220, 300), (1943, 300), 6163)

# Dense NN

In [8]:
output_columns = ['1024','2048','4096','8192']

#train_rds = train_rds.to_numpy()
train_y = train_mrs[output_columns].to_numpy()
#test_rds = test_rds.to_numpy()
test_y = test_mrs[output_columns].to_numpy()
x = torch.from_numpy(test_y)
torch.save(x, f'/kaggle/working/{model_name}_test_y.pt')

In [9]:
print(train_rds.shape, train_mrs.shape, train_embeddings.shape)
train_X = scaler.transform(train_rds)
test_X = scaler.transform(test_rds)
# train_X = np.append(scaler.transform(train_rds), train_embeddings, axis=1)
# test_X = np.append(scaler.transform(test_rds), test_embeddings, axis=1)
print(train_X.shape, test_X.shape)

(4220, 896) (4220, 4) (4220, 300)
(4220, 896) (1943, 896)


In [10]:
model = Sequential()
model.add(Dense(512, input_dim=train_X.shape[-1], activation='relu'))
model.add(Dense(len(output_columns), activation='sigmoid'))
model.compile(optimizer="adam", loss="mse")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               459264    
                                                                 
 dense_1 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 461,316
Trainable params: 461,316
Non-trainable params: 0
_________________________________________________________________


In [11]:
l = []
for i in range(50):
    history = model.fit(train_X, train_y, epochs=1, batch_size=16, validation_data=(test_X, test_y), verbose=1)
    y_pred = model(test_X)
    l.append(y_pred)
#     for i,j in enumerate(output_columns):
#         print('Test set MSE '+j+':',mean_squared_error(y_pred[:,i], test_y[:,i]))

264/264 [==============================] - 1s 4ms/step - loss: 0.0016 - val_loss: 0.0503


In [12]:
x = min(l, key=lambda x:mean_squared_error(x, test_y))
torch.save(x, f'/kaggle/working/{model_name}_dense.pt')

In [13]:
#test_rds = train_rds.reshape(test_rds.shape[0],1,896)
y_pred = model(test_X)
# print(test_X.shape)
# print(y_pred.shape)
for i,j in enumerate(output_columns):
    print(mean_squared_error(y_pred.numpy()[:,i], test_y[:,i]))
#print(mean_squared_error(y_pred.numpy()[:,1], test_mrs[:,1]))

0.017088801518760778
0.03765518886011031
0.07073043346278313
0.0758005319396419


# LSTM Network

In [14]:
output_columns = ['2048']
#train_rds = train_rds.to_numpy()
train_y = train_mrs[output_columns].to_numpy()
#test_rds = test_rds.to_numpy()
test_y = test_mrs[output_columns].to_numpy()

In [15]:
# print(train_rds.shape, train_mrs.shape, train_embeddings.shape)
train_X = np.reshape(scaler.transform(train_rds), (train_rds.shape[0], 1, 896))
test_X = np.reshape(scaler.transform(test_rds), (test_rds.shape[0], 1, 896))
train_emb = np.reshape(train_embeddings, (train_embeddings.shape[0], 1, 300))
test_emb = np.reshape(test_embeddings, (test_embeddings.shape[0], 1, 300))

print('train_X shape:',train_X.shape)
print('train_y shape:',train_y.shape)
print('train_e shape:',train_emb.shape)
print('~~~~~~~~')
print('test_X shape:',test_X.shape)
print('test_y shape:',test_y.shape)
print('test_e shape:',test_emb.shape)

train_X shape: (4220, 1, 896)
train_y shape: (4220, 1)
train_e shape: (4220, 1, 300)
~~~~~~~~
test_X shape: (1943, 1, 896)
test_y shape: (1943, 1)
test_e shape: (1943, 1, 300)


In [16]:
num_encoder_tokens = 300
num_decoder_tokens = 896
latent_dim = 1024

# Define the input sequence
encoder_inputs = Input(shape=(1, num_encoder_tokens))

# LSTM layer
encoder = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state
decoder_inputs = Input(shape=(1, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Attention layer
attention = Attention()
attention_output = attention([encoder_outputs, decoder_outputs])

# Dense layer
decoder_dense = Dense(len(output_columns), activation='sigmoid')
decoder_outputs = decoder_dense(attention_output)
print(decoder_outputs.shape)

# Define the model
model = Model([encoder_inputs, decoder_inputs], tf.squeeze(decoder_outputs, axis=1))
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error')


(None, 1, 1)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 300)]     0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 1, 1024),    5427200     ['input_1[0][0]']                
                                 (None, 1024),                                                    
                                 (None, 1024)]                                                    
                                                                                                  
 input_2 (InputLayer)           [(None, 1, 896)]     0           []                               
                                                                                 

In [17]:
# history = model.fit([train_emb, train_X], train_y, epochs=54, batch_size=64, validation_data=([test_emb, test_X], test_y), verbose=1)

In [18]:
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])

In [19]:
# y_prangle_spectrumed = model([test_emb, test_X])
# print(y_pred.shape)
# mean_squared_error(y_pred.numpy(), test_y)

In [20]:
# y_pred = model(test_X)
# for i,j in enumerate(output_columns):
#     print('Test set MSE:', mean_squared_error(y_pred.numpy(), test_y))

# RDs: CNN -> DNN, Embeddings:DNN

In [21]:
output_columns = ['1024', '2048', '4096', '8192']
train_y = train_mrs[output_columns].to_numpy()
test_y = test_mrs[output_columns].to_numpy()

scale = 0

if scale:
    train_rdss = scaler.transform(train_rds)
    test_rdss = scaler.transform(test_rds)
    train_embeddingss = embedding_scaler.transform(train_embeddings)
    test_embeddingss  = embedding_scaler.transform(test_embeddings)
else:
    train_rdss = train_rds.to_numpy()
    test_rdss = test_rds.to_numpy()
    train_embeddingss = train_embeddings
    test_embeddingss  = test_embeddings
    
train_X = np.reshape(train_rdss, (train_rds.shape[0], 896, 1))
test_X = np.reshape(test_rdss, (test_rds.shape[0], 896, 1))
train_emb = np.reshape(train_embeddingss, (train_embeddings.shape[0], 300))
test_emb = np.reshape(test_embeddingss, (test_embeddings.shape[0], 300))

print('train_X shape:',train_X.shape)
print('train_e shape:',train_emb.shape)
print('train_y shape:',train_y.shape)
print('~~~~~~~~')
print('test_X shape:',test_X.shape)
print('test_e shape:',test_emb.shape)
print('test_y shape:',test_y.shape)
print(train_emb[:5])

train_X shape: (4220, 896, 1)
train_e shape: (4220, 300)
train_y shape: (4220, 4)
~~~~~~~~
test_X shape: (1943, 896, 1)
test_e shape: (1943, 300)
test_y shape: (1943, 4)
[[ -0.585071   9.35283   41.856682 ...  -6.225739 -15.328747  14.726346]
 [ -0.585071   9.35283   41.856682 ...  -6.225739 -15.328747  14.726346]
 [ -0.585071   9.35283   41.856682 ...  -6.225739 -15.328747  14.726346]
 [ -0.585071   9.35283   41.856682 ...  -6.225739 -15.328747  14.726346]
 [ -0.585071   9.35283   41.856682 ...  -6.225739 -15.328747  14.726346]]


In [22]:
def create_model(att_nodes = 64, dnn_nodes = 1024):
    conv_out = 223
    
    embedding_inputs = Input(shape=(300))
    dense_emb = Dense(conv_out, activation='relu')
    embedding_outputs1 = dense_emb(embedding_inputs)
    
    emb_norm = tf.keras.layers.LayerNormalization(axis=-1)
    embedding_outputs1 = emb_norm(embedding_outputs1)
    
    expand_layer = tf.keras.layers.Reshape((conv_out,1))
    embedding_outputs = expand_layer(embedding_outputs1)
    
    print(embedding_outputs.shape)
    # CNN Layers
    hist_inputs = Input(shape=(896, 1))

    conv_layer = Conv1D(filters=6, kernel_size=8, strides=4)
    conv_outputs = conv_layer(hist_inputs)
    
    print(conv_outputs.shape)
#     pooling = tf.keras.layers.MaxPooling1D(pool_size=8, strides = 4)
#     conv_outputs = pooling(conv_outputs)

#     conv_outputs = tf.keras.layers.Flatten()(conv_outputs)

#     #Alternative
#     sec_dense = Dense(1024, activation='relu')
#     conv_outputs = sec_dense(tf.keras.layers.Flatten()(hist_inputs))
    
#     dense_hist = Dense(att_nodes, activation='relu')
#     hist_outputs = dense_hist(conv_outputs)
    
#     emb_norm = tf.keras.layers.LayerNormalization(axis=-1)
#     embedding_outputs = emb_norm(embedding_outputs)
    
#     hist_norm = tf.keras.layers.LayerNormalization(axis=-1)
#     hist_outputs = hist_norm(hist_outputs)
    
    hist_outputs = conv_outputs
    hist_norm = tf.keras.layers.LayerNormalization(axis=-2)
    hist_outputs = hist_norm(hist_outputs)
    
    # Attention layer
    attention = tf.keras.layers.Attention(score_mode='concat')
    attention_output = attention([embedding_outputs, hist_outputs])
    
    print(attention_output.shape)
    
    pooling = tf.keras.layers.MaxPooling1D(pool_size=8, strides = 4)
    attention_output = pooling(attention_output)

    attention_output = tf.keras.layers.Flatten()(attention_output)
    
    print(attention_output.shape)
    attention_output = tf.keras.layers.Concatenate()([attention_output, embedding_outputs1])
    print(attention_output.shape)

    
    drop_layer = tf.keras.layers.Dropout(0.2)
    attention_output = drop_layer(attention_output)
    
    att_norm = tf.keras.layers.LayerNormalization(axis=-1)
#     attention_output = att_norm(attention_output)
    
    dense_input = attention_output
    
    dense_layer1 = Dense(dnn_nodes, activation='relu')
    dense_output1 = dense_layer1(dense_input)
    
    dense_layer2 = Dense(len(output_columns), activation='sigmoid')
    total_output = dense_layer2(dense_output1)

    model = Model([hist_inputs, embedding_inputs], total_output)
    # model.summary()
    model.compile(optimizer="adam", loss="mse")
    y_pred = model([test_X, test_emb])
    print(y_pred.shape)
    print(mean_squared_error(y_pred, test_y))
    return model
create_model(128, 1024).summary()

(None, 223, 1)
(None, 223, 6)
(None, 223, 6)
(None, 324)
(None, 547)
(1943, 4)
0.12104782224444428
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 dense_3 (Dense)                (None, 223)          67123       ['input_3[0][0]']                
                                                                                                  
 input_4 (InputLayer)           [(None, 896, 1)]     0           []                               
                                                                                                  
 layer_normalization (LayerNorm  (None, 223)         446         ['dense_3[0][0]']          

In [23]:
# a = 512
# b = 0
# c = 1024
# mymodel = create_model(a, c)
# history = mymodel.fit([train_X, train_emb], train_y, epochs=60, batch_size=16, shuffle=True, validation_data=([test_X, test_emb], test_y))
# x = min(history.history['loss'])
# y = min(history.history['val_loss'])
# with open('/kaggle/working/metrics.txt', 'a') as f:
#     f.write(f'{a},{b},{c}:{x:.4f},{y:.4f}\n')
#     print(f'{a},{b},{c}:{x:.4f},{y:.4f}\n')

# RDs: CNN -> DNN, Embeddings:DNN

In [24]:
output_columns = ['1024', '2048', '4096', '8192']
train_y = train_mrs[output_columns].to_numpy()
test_y = test_mrs[output_columns].to_numpy()

scale = 0

if scale:
    train_rdss = scaler.transform(train_rds)
    test_rdss = scaler.transform(test_rds)
    train_embeddingss = embedding_scaler.transform(train_embeddings)
    test_embeddingss  = embedding_scaler.transform(test_embeddings)
else:
    train_rdss = train_rds.to_numpy()
    test_rdss = test_rds.to_numpy()
    train_embeddingss = train_embeddings
    test_embeddingss  = test_embeddings
    
train_X = np.reshape(train_rdss, (train_rds.shape[0], 896, 1))
test_X = np.reshape(test_rdss, (test_rds.shape[0], 896, 1))
train_emb = np.reshape(train_embeddingss, (train_embeddings.shape[0], 300))
test_emb = np.reshape(test_embeddingss, (test_embeddings.shape[0], 300))

print('train_X shape:',train_X.shape)
print('train_e shape:',train_emb.shape)
print('train_y shape:',train_y.shape)
print('~~~~~~~~')
print('test_X shape:',test_X.shape)
print('test_e shape:',test_emb.shape)
print('test_y shape:',test_y.shape)
print(train_emb[:5])

train_X shape: (4220, 896, 1)
train_e shape: (4220, 300)
train_y shape: (4220, 4)
~~~~~~~~
test_X shape: (1943, 896, 1)
test_e shape: (1943, 300)
test_y shape: (1943, 4)
[[ -0.585071   9.35283   41.856682 ...  -6.225739 -15.328747  14.726346]
 [ -0.585071   9.35283   41.856682 ...  -6.225739 -15.328747  14.726346]
 [ -0.585071   9.35283   41.856682 ...  -6.225739 -15.328747  14.726346]
 [ -0.585071   9.35283   41.856682 ...  -6.225739 -15.328747  14.726346]
 [ -0.585071   9.35283   41.856682 ...  -6.225739 -15.328747  14.726346]]


In [25]:
# Works  
def create_model(att_nodes = 64, dnn_nodes = 1024):
    
    embedding_inputs = Input(shape=(300))
    dense_emb = Dense(att_nodes, activation='relu')
    embedding_outputs = dense_emb(embedding_inputs)
    
#     expand_layer = tf.keras.layers.Reshape((conv_out,1))
#     embedding_outputs = expand_layer(embedding_outputs1)
    
    print(embedding_outputs.shape)
    # CNN Layers
    hist_inputs = Input(shape=(896, 1))

    conv_layer = Conv1D(filters=6, kernel_size=8, strides=4)
    conv_outputs = conv_layer(hist_inputs)
    
#     print(conv_outputs.shape)
    pooling = tf.keras.layers.MaxPooling1D(pool_size=8, strides = 4)
    conv_outputs = pooling(conv_outputs)

    conv_outputs = tf.keras.layers.Flatten()(conv_outputs)

#     #Alternative
#     sec_dense = Dense(1024, activation='relu')
#     conv_outputs = sec_dense(tf.keras.layers.Flatten()(hist_inputs))
    
    dense_hist = Dense(att_nodes, activation='relu')
    hist_outputs = dense_hist(conv_outputs)
    
    emb_norm = tf.keras.layers.LayerNormalization(axis=-1)
    embedding_outputs = emb_norm(embedding_outputs)
    
    hist_norm = tf.keras.layers.LayerNormalization(axis=-1)
    hist_outputs = hist_norm(hist_outputs)
    
#     hist_outputs = conv_outputs
#     hist_norm = tf.keras.layers.LayerNormalization(axis=-1)
#     hist_outputs = hist_norm(hist_outputs)
    
    
    attention_output = tf.keras.layers.Concatenate()([hist_outputs, embedding_outputs])

    
    drop_layer = tf.keras.layers.Dropout(0.2)
    attention_output = drop_layer(attention_output)
    
    att_norm = tf.keras.layers.LayerNormalization(axis=-1)
    dense_input = attention_output # att_norm(attention_output)
    
    dense_layer1 = Dense(dnn_nodes, activation='relu')
    dense_output1 = dense_layer1(dense_input)
    
    dense_layer2 = Dense(len(output_columns), activation='sigmoid')
    total_output = dense_layer2(dense_output1)

    model = Model([hist_inputs, embedding_inputs], total_output)
    # model.summary()
    model.compile(optimizer="adam", loss="mse")
    y_pred = model([test_X, test_emb])
    print(y_pred.shape)
    print(mean_squared_error(y_pred, test_y))
    return model
create_model(128, 1024).summary()

(None, 128)
(1943, 4)
0.0973102381078598
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 896, 1)]     0           []                               
                                                                                                  
 conv1d_1 (Conv1D)              (None, 223, 6)       54          ['input_6[0][0]']                
                                                                                                  
 max_pooling1d_1 (MaxPooling1D)  (None, 54, 6)       0           ['conv1d_1[0][0]']               
                                                                                                  
 flatten_1 (Flatten)            (None, 324)          0           ['max_pooling1d_1[0][0]']        
                                                   

In [26]:
a = 512
b = 0
c = 1024
mymodel = create_model(a, c)
l = []
for i in range(50):
    mymodel.fit([train_X, train_emb], train_y, epochs=1, batch_size=16, shuffle=True, validation_data=([test_X, test_emb], test_y))
    l.append(mymodel([test_X, test_emb]))
# x = min(history.history['loss'])
# y = min(history.history['val_loss'])

(None, 512)
(1943, 4)
0.23288575225009156
264/264 [==============================] - 2s 6ms/step - loss: 6.0969e-04 - val_loss: 0.0446


In [27]:
y_pred = mymodel([test_X, test_emb])
print(y_pred.shape)
print(mean_squared_error(y_pred, test_y))

(1943, 4)
0.04456642970526753


In [28]:
mean_squared_error(l[3], test_y)

0.04429682746418989

In [29]:
print(mean_squared_error(min(l, key=lambda x:mean_squared_error(x, test_y)), test_y))

0.037132853703101


In [30]:
x = min(l, key=lambda x:mean_squared_error(x, test_y))
torch.save(x, f'/kaggle/working/{model_name}_big.pt')

In [31]:
f = torch.load(f'/kaggle/working/{model_name}_big.pt')

In [32]:
f.shape

TensorShape([1943, 4])

In [33]:
x = torch.from_numpy(test_y)
# torch.save(x, '/kaggle/working/test_y.pt')
print(x.shape)
print(mean_squared_error(x,f))

torch.Size([1943, 4])
0.037132853703101


# RDs: LSTM -> DNN, Embeddings:DNN

In [34]:
output_columns = ['1024', '2048', '4096', '8192']
train_y = train_mrs[output_columns].to_numpy()
test_y = test_mrs[output_columns].to_numpy()

scale = 1

if scale:
    train_rdss = scaler.transform(train_rds)
    test_rdss = scaler.transform(test_rds)
    train_embeddingss = embedding_scaler.transform(train_embeddings)
    test_embeddingss  = embedding_scaler.transform(test_embeddings)
else:
    train_rdss = train_rds.to_numpy()
    test_rdss = test_rds.to_numpy()
    train_embeddingss = train_embeddings
    test_embeddingss  = test_embeddings
    
train_X = np.reshape(train_rdss, (train_rds.shape[0], 896, 1))
test_X = np.reshape(test_rdss, (test_rds.shape[0], 896, 1))
train_emb = np.reshape(train_embeddingss, (train_embeddings.shape[0], 300))
test_emb = np.reshape(test_embeddingss, (test_embeddings.shape[0], 300))

print('train_X shape:',train_X.shape)
print('train_e shape:',train_emb.shape)
print('train_y shape:',train_y.shape)
print('~~~~~~~~')
print('test_X shape:',test_X.shape)
print('test_e shape:',test_emb.shape)
print('test_y shape:',test_y.shape)
print(train_emb[:5])

train_X shape: (4220, 896, 1)
train_e shape: (4220, 300)
train_y shape: (4220, 4)
~~~~~~~~
test_X shape: (1943, 896, 1)
test_e shape: (1943, 300)
test_y shape: (1943, 4)
[[ 0.09458411 -0.25581078 -0.32419651 ...  0.28993452  0.32376586
  -0.33153817]
 [ 0.09458411 -0.25581078 -0.32419651 ...  0.28993452  0.32376586
  -0.33153817]
 [ 0.09458411 -0.25581078 -0.32419651 ...  0.28993452  0.32376586
  -0.33153817]
 [ 0.09458411 -0.25581078 -0.32419651 ...  0.28993452  0.32376586
  -0.33153817]
 [ 0.09458411 -0.25581078 -0.32419651 ...  0.28993452  0.32376586
  -0.33153817]]


In [35]:
train_X_rev = np.flip(train_X, axis=1)
test_X_rev = np.flip(test_X, axis=1)

In [36]:
def create_lstm_model(att_nodes = 64, dnn_nodes = 1024):
    
    embedding_inputs = Input(shape=(300))
    dense_emb = Dense(att_nodes, activation='relu')
    embedding_outputs = dense_emb(embedding_inputs)
    
#     expand_layer = tf.keras.layers.Reshape((conv_out,1))
#     embedding_outputs = expand_layer(embedding_outputs1)
    
    print(embedding_outputs.shape)
    # CNN Layers
    hist_inputs = Input(shape=(896, 1))
    
    lstm1 = LSTM(16, return_sequences=True, return_state=True)
    lstm_output1, state_h, state_c = lstm1(hist_inputs)
    
    encoder_states = [state_h, state_c]
    
    rev_inputs = Input(shape=(896, 1))
    lstm2 = LSTM(16, return_sequences=True)
    lstm_output2 = lstm2(rev_inputs, initial_state=encoder_states)
    
#     print(lstm_output1.shape, lstm_output1)
    
#     attention = Attention()
#     attention_output = attention([lstm_output1, lstm_output2])
    
    attention_output = tf.keras.layers.Concatenate()([lstm_output1, lstm_output2])

    lstm_outputs = tf.keras.layers.Flatten()(attention_output)


    dense_hist = Dense(att_nodes, activation='relu')
    hist_outputs = dense_hist(lstm_outputs)
    
    emb_norm = tf.keras.layers.LayerNormalization(axis=-1)
    embedding_outputs = emb_norm(embedding_outputs)
    
    hist_norm = tf.keras.layers.LayerNormalization(axis=-1)
    hist_outputs = hist_norm(hist_outputs)
    
    attention_output = tf.keras.layers.Concatenate()([hist_outputs, embedding_outputs])

    
    drop_layer = tf.keras.layers.Dropout(0.2)
    attention_output = drop_layer(attention_output)
    
    att_norm = tf.keras.layers.LayerNormalization(axis=-1)
    dense_input = attention_output # att_norm(attention_output)
    
    dense_layer1 = Dense(dnn_nodes, activation='relu')
    dense_output1 = dense_layer1(dense_input)
    
    dense_layer2 = Dense(len(output_columns), activation='sigmoid')
    total_output = dense_layer2(dense_output1)

    model = Model([hist_inputs, rev_inputs, embedding_inputs], total_output)
    # model.summary()
    model.compile(optimizer="adam", loss="mse")
    y_pred = model([test_X, test_X_rev, test_emb])
    print(y_pred.shape)
    print(mean_squared_error(y_pred, test_y))
    return model
create_lstm_model(128, 1024).summary()

(None, 128)
(1943, 4)
0.1422855789969752
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 896, 1)]     0           []                               
                                                                                                  
 lstm_2 (LSTM)                  [(None, 896, 16),    1152        ['input_10[0][0]']               
                                 (None, 16),                                                      
                                 (None, 16)]                                                      
                                                                                                  
 input_11 (InputLayer)          [(None, 896, 1)]     0           []                               
                                                   

In [37]:
a = 256
b = 0
c = 1024
mymodel = create_lstm_model(a, c)
l = []
for i in range(50):
    mymodel.fit([train_X, train_X_rev, train_emb], train_y, epochs=1, batch_size=16, shuffle=True, validation_data=([test_X, test_X_rev, test_emb], test_y))
    l.append(mymodel([test_X, test_X_rev, test_emb]))

(None, 256)
(1943, 4)
0.1340275512834403
264/264 [==============================] - 15s 58ms/step - loss: 4.2117e-04 - val_loss: 0.0428


In [38]:
x = min(l, key=lambda x:mean_squared_error(x, test_y))
torch.save(x, f'/kaggle/working/{model_name}_lstm.pt')

In [39]:
f = torch.load( f'/kaggle/working/{model_name}_lstm.pt')
print(mean_squared_error(f, test_y))

0.03707690081494065
